# Import all the necessary libraries

In [1]:
import pandas as pd
import glob

# Merge all cleaned csv files into a single csv file

In [2]:
# Specify the path to your stock CSV files
stock_files_path = '../data/cleaned_stock_data/*.csv'

# Use glob to find all the CSV files in the specified directory
stock_files = glob.glob(stock_files_path)

# Read each CSV file and store the DataFrames in a list
stock_dfs = [pd.read_csv(file) for file in stock_files]

# Concatenate all DataFrames into one
combined_stock_df = pd.concat(stock_dfs, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_stock_df.to_csv('all_stock_data.csv', index=False)

print("All stock data has been successfully merged into all_stock_data.csv")

All stock data has been successfully merged into all_stock_data.csv
